In [1]:
import re
import cv2
import math

"""
 to install tflite_runtime use the command pip install --extra-index-url https://google-coral.github.io/py-repo/ tflite_runtime"
"""
from tflite_runtime.interpreter import Interpreter
import numpy as np

In [2]:
CAMERA_WIDTH = 640
CAMERA_HEIGHT = 480 # this dims is for webcam


In [3]:
def load_labels(path='custom_model_lite\labelmap.txt'):
  """Loads the labels file. Supports files with or without index numbers."""
  with open(path, 'r', encoding='utf-8') as f:
    lines = f.readlines()
    labels = {}
    for row_number, content in enumerate(lines):
      pair = re.split(r'[:\s]+', content.strip(), maxsplit=1)
      if len(pair) == 2 and pair[0].strip().isdigit():
        labels[int(pair[0])] = pair[1].strip()
      else:
        labels[row_number] = pair[0].strip()
  return labels

In [4]:
def set_input_tensor(interpreter, image):
  """Sets the input tensor."""
  tensor_index = interpreter.get_input_details()[0]['index']
  input_tensor = interpreter.tensor(tensor_index)()[0]
  input_tensor[:, :] = np.expand_dims((image-255)/255, axis=0)

In [5]:
def get_output_tensor(interpreter, index):
  """Returns the output tensor at the given index."""
  output_details = interpreter.get_output_details()[index]
  tensor = np.squeeze(interpreter.get_tensor(output_details['index']))
  return tensor

In [6]:
def detect_objects(interpreter, image, threshold):
  """Returns a list of detection results, each a dictionary of object info."""
  set_input_tensor(interpreter, image)
  interpreter.invoke()
  # Get all output details
  scores = get_output_tensor(interpreter, 0)
  boxes = get_output_tensor(interpreter, 1)
  count = int(get_output_tensor(interpreter, 2))
  classes = get_output_tensor(interpreter, 3)
  results = []
  for i in range(count):
    if scores[i] >= threshold:
      result = {
          'bounding_box': boxes[i],
          'class_id': classes[i],
          'score': scores[i]
      }
      results.append(result)
  return results

In [10]:
labels = load_labels()
interpreter = Interpreter('custom_model_lite(1)\detect.tflite')
interpreter.allocate_tensors()
_, input_height, input_width, _ = interpreter.get_input_details()[0]['shape']

cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    img = cv2.resize(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB), (320,320))
    res = detect_objects(interpreter, img, 0.5)

    for result in res:
        print(1)
        ymin, xmin, ymax, xmax = result['bounding_box']
        xmin = int(max(1,xmin * CAMERA_WIDTH))
        xmax = int(min(CAMERA_WIDTH, xmax * CAMERA_WIDTH))
        ymin = int(max(1, ymin * CAMERA_HEIGHT))
        ymax = int(min(CAMERA_HEIGHT, ymax * CAMERA_HEIGHT))
        
        cv2.rectangle(frame,(xmin, ymin),(xmax, ymax),(0,255,0),3)
        cv2.putText(frame,labels[int(result['class_id'])]+" "+str(round(result['score'],2)),(xmin, min(ymax, CAMERA_HEIGHT-20)), cv2.FONT_HERSHEY_SIMPLEX, 0.5,(255,255,255),2,cv2.LINE_AA) 

    cv2.imshow('Pi Feed', frame)

    if cv2.waitKey(10) & 0xFF ==ord('q'):
        cap.release()
        cv2.destroyAllWindows()

1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


In [58]:
cap.release()
cv2.destroyAllWindows()

In [21]:
from ultralytics import YOLO
import cv2
import numpy as np
import cvzone
import math
import mediapipe as mp
import time
import torch as th

In [19]:
model = YOLO('bestnano.pt')


caminput =1

In [20]:
cap = cv2.VideoCapture(0)
## phone detection 
while True:
    ret,frame = cap.read()
    frame = cv2.flip(frame,1)
    res = model(frame,stream=True,show=False)

    for r in res:
        boxes = r.boxes
        for box in boxes:
            ## phone index is 67
            clas= int(box.cls[0])
            # if clas != 67:
            #     continue
            x1,y1,x2,y2 = box.xyxy[0]
            x1,y1,x2,y2 = int(x1),int(y1),int(x2),int(y2)
            # cv2.rectangle(frame,(x1,y1),(x2,y2),(0,255,0),2)
            bbox = (x1,y1,x2-x1,y2-y1)
            cvzone.cornerRect(frame,bbox)
            cv2.putText(frame,"smoking"+str(float(box.conf[0])),(x1,y1-10),cv2.FONT_HERSHEY_COMPLEX,1,(0,255,0),2)
    cv2.imshow('frame',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

In [6]:
cap.release()
cv2.destroyAllWindows()